### bloc de préparation

In [1]:
%matplotlib inline

# ici on met les librairies à importer
import numpy as np
import pandas as pd
import pylab as P

from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier 
#from sklearn.ensemble import DecisionTreeClassifier

import matplotlib.pyplot as plt

from sklearn.cross_validation import train_test_split

In [2]:
### voici les variables globales ###

#les données a charger. index_col=[0] permet d'avoir la colonne "PassengerId" comme index de table 
train_df = pd.read_csv('train.csv', header=0,index_col=[0])
#train_df = pd.read_csv('tempo.csv', header=0,index_col=[0])
test_df = pd.read_csv('test.csv', header=0,index_col=[0])



In [3]:
#X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.25,random_state=11)
#train, test_cible = train_test_split(train_df, train_size = 0.8)
#train = train.copy()
#test_cible = test_cible.copy()
#test = test_cible.drop(['Survived'], axis=1).copy()

#pour faire le test kaggle il faut décommenter les lignes suivantes
train = train_df
test = test_df
test_cible = test_df

In [18]:
train_df[["Sex", "Survived"]].groupby(['Sex']).mean()

,Survived
Sex,
female,0.742038
male,0.188908


In [19]:
train_df[["Pclass", "Survived"]].groupby(['Pclass']).mean()

,Survived
Pclass,
1,0.629630
2,0.472826
3,0.242363


In [5]:
# on s'occupe de créer la colonne gender female: 0, male: 1
train['Gender'] = train['Sex'].map( {'female': 0, 'male': 1} ).astype(int)
#train_df['Gender'] = train_df['Sex'].map( {'female': 0, 'male': 1} ).astype(int)

test['Gender'] = test['Sex'].map( {'female': 0, 'male': 1} ).astype(int)
#test_df['Gender'] = test_df['Sex'].map( {'female': 0, 'male': 1} ).astype(int)



In [10]:
# il faut remplir les ages non renseignés
# etape 1 on cree une colonne AgeFill
train['AgeFill'] = train['Age']
test['AgeFill'] = test['Age']
# Etape 2 on remplit AgeFill avec l'age median en fonction de la classe et du sexe

for genre in range(0, 2):  # on selectionne le sexe (0: female, 1:male)
    for classe in range(0, 3):  #on selectionne la classe 
        train.loc[(train_df.Age.isnull()) & (train.Gender == genre ) & \
        (train.Pclass == classe + 1), 'AgeFill'] = \
        train[(train['Gender'] == genre ) & (train['Pclass'] == classe + 1)]['Age'].dropna().median()  
        
for genre in range(0, 2):
    for classe in range(0, 3):
        test.loc[(test.Age.isnull()) & (test.Gender == genre ) & \
        (test.Pclass == classe + 1 ), 'AgeFill'] = \
        test[(test['Gender'] == genre ) & (test['Pclass'] == classe + 1)]['Age'].dropna().median()
        
train['AgeFill'] = train['AgeFill'].astype(int)
test['AgeFill'] = test['AgeFill'].astype(int)

In [11]:
# on convertit Embarked en int 

train['Embarked_Fill'] = train['Embarked'].astype(str)
test['Embarked_Fill'] = test['Embarked'].astype(str)
#train_df['Embarked_Fill'].unique()
train['Embarked_Fill'] = train['Embarked_Fill'].map( {'S': 0, 'C': 1, 'Q': 2, 'nan': 0} ).astype(int)
test['Embarked_Fill'] = test['Embarked_Fill'].map( {'S': 0, 'C': 1, 'Q': 2, 'nan': 0} ).astype(int)

In [12]:
#travail sur le prix du billet en fonction de la classe

train_df['Fare_Fill'] = (train_df['Fare']//10).clip_upper(3).astype(int)

test_df['Fare_Fill'] = (test_df['Fare']//10).clip_upper(3).fillna(3-test_df['Pclass']).astype(int)

for i in train_df['Fare_Fill'].unique():
    print i, len(train_df[( train_df['Fare_Fill'] == i )])

0 336
3 240
2 136
1 179


In [13]:
# on regarde les personnes ayant une famille

train['Family'] =  train["Parch"] + train["SibSp"]
train['Family'].loc[train['Family'] > 0] = 1
train['Family'].loc[train['Family'] == 0] = 0

test['Family'] =  test["Parch"] + test["SibSp"]
test['Family'].loc[test['Family'] > 0] = 1
test['Family'].loc[test['Family'] == 0] = 0

/Users/philippe/anaconda/lib/python2.7/site-packages/pandas/core/indexing.py:128: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [16]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 418 entries, 892 to 1309
Data columns (total 7 columns):
Pclass       418 non-null int64
SibSp        418 non-null int64
Parch        418 non-null int64
Gender       418 non-null int64
AgeFill      418 non-null int64
Fare_Fill    418 non-null int64
Family       418 non-null int64
dtypes: int64(7)
memory usage: 26.1 KB


# machine learning

In [15]:
# il faut supprimer les colonnes inutiles
test = test.drop(['Name', 
                  'Sex', 
                  'Age', 
                  'Ticket', 
                  'Cabin',
                  'Embarked',
                  'Embarked_Fill',
                  'Fare'], axis=1) 


train = train.drop(['Name',
                    'Sex', 
                    'Age',
                    'Ticket', 
                    'Cabin',
                    'Embarked',
                    'Embarked_Fill',
                    'Fare'], axis=1)

In [17]:
train_data = train.values
test_data = test.values

# Entrainement

#model = AdaBoostClassifier(n_estimators= 500)
model = AdaBoostClassifier(
                n_estimators=500
                #,learning_rate=0.1
)

model = model.fit( train_data[0::,1::], train_data[0::,0] )

# Prediction

output = model.predict(test_data).astype(int)

In [18]:
#test_cible['Survived2'] = output
test_cible['Survived'] = output
test_cible[['Survived']].to_csv('resultat.csv')

# randomforest 

In [20]:
train_data = train.values
test_data = test.values

# Create the random forest object which will include all the parameters
# for the fit
forest = RandomForestClassifier(n_estimators = 100)

# Fit the training data to the Survived labels and create the decision trees
forest = forest.fit(train_data[0::,1::],train_data[0::,0])

# Take the same decision trees and run it on the test data
output = forest.predict(test_data).astype(int)

In [21]:
test_cible['Survived'] = output
test_cible[['Survived']].to_csv('resultat.csv')

In [ ]:
test_cible[['Survived']]